In [1]:
%%capture
! pip install xclim

In [ ]:
import xclim.sdba

In [ ]:
import xesmf
import xarray as xr
import numpy as np
from science_validation_manual import *
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import rhg_compute_tools.kubernetes as rhgk

In [ ]:
client, cluster = rhgk.get_micro_cluster()

In [ ]:
cluster.scale(100)
cluster

In [ ]:
domain = read_gcs_zarr('gs://support-c23ff1a3/domain.0p25x0p25.zarr')

In [ ]:
domain

In [ ]:
ds_before_regrid = read_gcs_zarr('gs://scratch-170cd6ec/d9d8e40f-8b68-477b-a706-ed2e2a21698e/e2e-bcc-csm2-mr-pr-dev-xkhlh-4082033234/rechunked.zarr')

In [ ]:
regridder = xesmf.Regridder(
    x,
    domain,
    method='conservative'
)

In [ ]:
client.close()
cluster.close()

In [ ]:
da = read_gcs_zarr('gs://clean-b1dbca25/cmip6/ScenarioMIP/BCC/BCC-CSM2-MR/ssp370/r1i1p1f1/day/pr/gn/v20190318.zarr')['pr']
plt.figure(figsize=(24,24))
ax = plt.axes(projection=ccrs.PlateCarree())
da.isel(time=slice(0, 720)).where(da < 1).count('time').plot.pcolormesh(ax=ax);
ax.coastlines();
plt.title(f'dry days bcc-csm2 ssp370 cleaned')

In [ ]:
da = read_gcs_zarr('gs://scratch-170cd6ec/c9d92aef-c68c-479b-9ebd-cd1d643000b9/qdm-preprocess-dev-6ksl6-1876554127/out.zarr')['pr']
plt.figure(figsize=(24,24))
ax = plt.axes(projection=ccrs.PlateCarree())
da.where(da <= 1).count('time').plot.pcolormesh(ax=ax);
ax.coastlines();
plt.title(f'dry days bcc-csm2 ssp370 non wd and regridded')

In [ ]:
da = read_gcs_zarr('gs://scratch-170cd6ec/c9d92aef-c68c-479b-9ebd-cd1d643000b9/qdm-preprocess-dev-6ksl6-614060684/rechunked.zarr')

In [ ]:
da.where(da < 1).min().compute()

In [ ]:
da.where(da < 1).max().compute()

In [ ]:
athens = da.sel(lon=37.98, lat=23.72, method='nearest')

In [ ]:
athens_dry = athens.where(athens<1, drop=True)

In [ ]:
minval = da.min().compute()

In [ ]:
da = read_gcs_zarr('gs://scratch-170cd6ec/d9d8e40f-8b68-477b-a706-ed2e2a21698e/e2e-bcc-csm2-mr-pr-dev-xkhlh-2016165429/wdf-corrected.zarr')['pr']
plt.figure(figsize=(24,24))
ax = plt.axes(projection=ccrs.PlateCarree())
da.where(da == minval.item()).count('time').plot.pcolormesh(ax=ax);
ax.coastlines();
plt.title(f'at threhsold bcc-csm2 ssp370 wd NOT regridded, e2e-bcc-csm2-mr-pr-dev-xkhlh')

In [ ]:
da = read_gcs_zarr('gs://scratch-170cd6ec/d9d8e40f-8b68-477b-a706-ed2e2a21698e/e2e-bcc-csm2-mr-pr-dev-xkhlh-1521363201/out.zarr')['pr']
plt.figure(figsize=(24,24))
ax = plt.axes(projection=ccrs.PlateCarree())
da.isel(time=slice(0, 720)).where(da == da.min().compute().values.item()).count('time').plot.pcolormesh(ax=ax);
ax.coastlines();
plt.title(f'== 0.6268846 days bcc-csm2 ssp370 wd and regridded, e2e-bcc-csm2-mr-pr-dev-xkhlh')

In [ ]:
da = read_gcs_zarr('gs://scratch-170cd6ec/c9d92aef-c68c-479b-9ebd-cd1d643000b9/qdm-preprocess-dev-6ksl6-1876554127/out.zarr')['pr']
plt.figure(figsize=(24,24))
ax = plt.axes(projection=ccrs.PlateCarree())
da.where(da < 0.7).count('time').plot.pcolormesh(ax=ax);
ax.coastlines();
plt.title(f'< 0.7 days bcc-csm2 ssp370 NO wd and regridded, qdm-preprocess-dev-6ksl6')

In [ ]:
da = read_gcs_zarr('gs://scratch-170cd6ec/51f4590d-9c75-4682-80d7-74090e4b3072/qdm-preprocess-dev-sk77z-2311758123/out.zarr')['pr']
plt.figure(figsize=(24,24))
ax = plt.axes(projection=ccrs.PlateCarree())
da.where(da < 1.67).count('time').plot.pcolormesh(ax=ax);
ax.coastlines();
plt.title(f'< 0.7 days bcc-csm2 ssp370 wd at 2 and regridded, qdm-preprocess-dev-sk77z')

In [ ]:
#da = xr.DataArray(data=np.array([[0.5, 1],[3, 0.2]]), dims=['lon','lat'], coords={'lon':[40, 50], 'lat':[30, 20]})
def censor(x):
    if x < 1:
        return np.random.uniform(low=0.5, high=1)
    else:
        return x

In [ ]:
ds = read_gcs_zarr('gs://clean-b1dbca25/cmip6/ScenarioMIP/BCC/BCC-CSM2-MR/ssp370/r1i1p1f1/day/pr/gn/v20190318.zarr')

In [ ]:
ds = ds.drop_dims('bnds')

In [ ]:
ds = ds.isel(time=slice(0, 720))

In [ ]:
ds = ds.compute()

In [ ]:
def censor(x):
    if x < 1:
        return np.random.uniform(low=0.5, high=1)
    else:
        return x

In [ ]:
# [-180, 180] -> [0, 360]
def lon360(lon180):
    return ((360 + (lon180 % 360)) % 360)
lon360 = np.vectorize(lon360)

In [ ]:
# vice versa
def lon180(lon360):
    if (lon360 > 180.0):
        return lon360 - 360.0
    else:
        return lon360
lon180 = np.vectorize(lon180)

In [ ]:
#ds.lon_bnds.values = lon180(ds.lon_bnds.values)

In [ ]:
#ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180))

In [ ]:
ds_corrected = xr.apply_ufunc(np.vectorize(censor), ds)

In [ ]:
# wet day frequency correction
#ds_corrected = ds.where(ds > 1.0, np.random.uniform(low=0.5, high=1.0))

In [ ]:
ds_corrected['pr'].min().values.item()

In [ ]:
# regridding
domain = read_gcs_zarr('gs://support-c23ff1a3/domain.1x1.zarr')

In [ ]:
domain

In [ ]:
#domain = xesmf.util.grid_global(1, 1, cf=True)

In [ ]:
#domain = domain.rename({"lon":"longitude", "lat":"latitude"})

In [ ]:
#ds_corrected = ds_corrected.rename({"lon":"longitude", "lat":"latitude"})

In [ ]:
ds_corrected

In [ ]:
regridder = xesmf.Regridder(ds_in=ds_corrected, ds_out=domain, method='bilinear')

In [ ]:
regridded = regridder(indata=ds_corrected['pr'])

In [ ]:
da = ds_corrected['pr']
plt.figure(figsize=(24,24))
ax = plt.axes(projection=ccrs.PlateCarree())
da.isel(time=0).plot.pcolormesh(ax=ax);
ax.coastlines();
plt.title(f'before')

In [ ]:
da = regridded
plt.figure(figsize=(24,24))
ax = plt.axes(projection=ccrs.PlateCarree())
da.where(da<1).count('time').plot.pcolormesh(ax=ax);
ax.coastlines();
plt.title(f'after')

In [ ]:
# GCM cleaned data
ds

In [ ]:
# 'gs://support-c23ff1a3/domain.1x1.zarr'
domain

In [ ]:
ds_corrected.lon_bnds.values

In [ ]:
import xesmf as xe

# target grid (2 times 2 cells)
coarse_grid = xe.util.grid_global(0.25, 0.25)#xe.util.grid_global(180, 90)
# source grid (3 times 3 cells) with some array of values
fine_grid = xe.util.grid_global(1, 1)#xe.util.grid_global(72, 36)
da = xe.data.wave_smooth(fine_grid["lon"], fine_grid["lat"])
#da.values = np.ones((5, 5))
#da.values = np.array([[0.05, 0.2, 0.05], [0.1, 0.1, 0.1], [0.3, 0.05, 0.05]])*100
fine_grid['data'] = da
regridder = xe.Regridder(ds_in=fine_grid, ds_out=coarse_grid, method='conservative')
regridded = regridder(fine_grid['data'])

In [ ]:
da = regridded
plt.figure(figsize=(24,24))
ax = plt.axes(projection=ccrs.PlateCarree())
da.plot.pcolormesh(ax=ax);
ax.coastlines();
plt.title(f'after')